In [1]:
# Cài đặt thư viện cần thiết
!pip install -q ultralytics kagglehub scikit-learn seaborn transformers
!pip install "numpy<2.0" "scipy<1.14" "matplotlib>=3.8,<3.9" seaborn scikit-learn

# Import các thư viện
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
# BỎ import torchvision ViT
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import sys
import yaml
import math
import logging
import kagglehub
from tqdm.notebook import tqdm
from pathlib import Path
from ultralytics import YOLO

# (SỬA ĐỔI) Import đúng model ViT từ transformers
from transformers import ViTImageProcessor, ViTForImageClassification

# Import các hàm tính toán metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, f1_score, roc_auc_score
from scipy.optimize import brentq
from scipy.interpolate import interp1d

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(message)s')
logger = logging.getLogger(__name__)

# Thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 93.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

E0000 00:00:1761918936.548435      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761918936.599963      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Sử dụng thiết bị: cuda


In [2]:
# Hàm tiện ích để nạp checkpoint (SỬA LỖI: dùng strict=True)
def load_checkpoint_to_model(model_instance: torch.nn.Module, checkpoint_path: str, device=None):
    device = device or torch.device('cpu')
    if not os.path.isfile(checkpoint_path):
        raise FileNotFoundError(f"File checkpoint không tồn tại: {checkpoint_path}")
        
    logger.info(f"Đang nạp checkpoint vào model từ: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    state_dict = checkpoint.get('model_state_dict', checkpoint)

    first_key = next(iter(state_dict.keys()), None)
    if first_key is None:
        raise ValueError(f"State dict rỗng trong checkpoint: {checkpoint_path}")
        
    is_data_parallel = first_key.startswith('module.')

    if is_data_parallel:
        logger.info("Phát hiện state_dict từ DataParallel, đang loại bỏ tiền tố 'module.'...")
        state_dict = {k.replace('module.', '', 1): v for k, v in state_dict.items()}
    
    # (SỬA LỖI) Dùng strict=True để đảm bảo nạp đúng kiến trúc
    load_result = model_instance.load_state_dict(state_dict, strict=True) 
    logger.info(f"Kết quả load state dict (strict=True): {load_result}") 

    # model_instance = torch.nn.DataParallel(model_instance)
    model_instance.to(device)
    model_instance.eval()
    logger.info("Nạp checkpoint thành công.")
    return model_instance

print("Đã định nghĩa hàm load_checkpoint_to_model (strict=True).")

Đã định nghĩa hàm load_checkpoint_to_model (strict=True).


In [3]:
# --- Đăng nhập Kaggle Hub ---
try:
    print("Đang đăng nhập Kaggle Hub...")
    kagglehub.login()
    print("Kaggle Hub login thành công.")
except Exception as e:
    logger.warning(f"Kaggle Hub login lỗi/bỏ qua: {e}.")

# --- Định nghĩa Handle ---
YOLO_HANDLE = "khnhnguyn222/yolo-facedetection/pyTorch/default"
VIT_HANDLE = "khnhnguyn222/vision-transformer/pytorch/default/1"

# (SỬA LỖI) SỬA TÊN FILE THÀNH CHECKPOINT ĐÚNG
VIT_FILENAME = "ViT.pt" # Tên file 97.74% Acc

NUM_VIT_CLASSES = 2 
MODEL_STR = "google/vit-base-patch16-224-in21k" # Dùng để lấy kiến trúc gốc

# --- Tải Model YOLO ---
try:
    print(f"Đang tải YOLO từ: {YOLO_HANDLE}")
    yolo_model_dir = kagglehub.model_download(YOLO_HANDLE)
    yolo_model_path = os.path.join(yolo_model_dir, "YOLO.pt") 
    if not os.path.exists(yolo_model_path):
        pt_files = [f for f in os.listdir(yolo_model_dir) if f.endswith(".pt")]
        yolo_model_path = os.path.join(yolo_model_dir, pt_files[0])
    yolo_model = YOLO(yolo_model_path)
    # yolo_model = torch.nn.DataParallel(yolo_model)
    yolo_model.to(device) 
    print(f"Tải YOLO model thành công từ: {yolo_model_path}")
except Exception as e:
    logger.error(f"Lỗi khi tải YOLO: {e}"); raise e

# --- (SỬA LỖI KIẾN TRÚC) Tải Model ViT (PAD) ---
try:
    print(f"Đang tải ViT từ: {VIT_HANDLE} (file: {VIT_FILENAME})")
    vit_model_dir = kagglehub.model_download(VIT_HANDLE)
    vit_model_path = os.path.join(vit_model_dir, VIT_FILENAME) 
    if not os.path.exists(vit_model_path):
         raise FileNotFoundError(f"Không tìm thấy file {VIT_FILENAME} trong thư mục ViT. Hãy kiểm tra lại tên file trên Kaggle Hub.")

    # Khởi tạo kiến trúc HUGGING FACE (giống lúc train)
    vit_model = ViTForImageClassification.from_pretrained(
        MODEL_STR, 
        num_labels=NUM_VIT_CLASSES 
    )
    
    # Load state_dict từ checkpoint .pt đã train
    load_checkpoint_to_model(vit_model, vit_model_path, device=device)
    print(f"Tải ViT (PAD) model (Transformers) thành công từ: {vit_model_path}")
    
except Exception as e:
    logger.error(f"Lỗi khi tải ViT: {e}")
    raise e

print("\n--- Tải tất cả Model cho đánh giá PAD thành công! ---")

Đang đăng nhập Kaggle Hub...


Kaggle Hub login thành công.
Đang tải YOLO từ: khnhnguyn222/yolo-facedetection/pyTorch/default
Tải YOLO model thành công từ: /kaggle/input/yolo-facedetection/pytorch/default/1/YOLO.pt
Đang tải ViT từ: khnhnguyn222/vision-transformer/pytorch/default/1 (file: ViT.pt)


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tải ViT (PAD) model (Transformers) thành công từ: /kaggle/input/vision-transformer/pytorch/default/1/ViT.pt

--- Tải tất cả Model cho đánh giá PAD thành công! ---


In [4]:
# Lớp Dataset cho CelebA-Spoof
class CelebASpoofDataset(Dataset):
    def __init__(self, root_dir, meta_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform 
        self.samples = []
        with open(meta_file, "r") as f:
            for line in f:
                path, label = line.strip().split()
                img_path = os.path.join(root_dir, path)
                label = int(label)
                if os.path.exists(img_path):
                    self.samples.append((img_path, label))
        print(f"Loaded {len(self.samples)} samples from {meta_file}")
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]

print("Đã định nghĩa CelebASpoofDataset.")

# --- (SỬA LỖI) Lấy Processor/Transform đúng (giống lúc train) ---
try:
    processor = ViTImageProcessor.from_pretrained(MODEL_STR)
    logger.info(f"Đã tải processor từ {MODEL_STR}")
except Exception as e:
    logger.error(f"Không thể tải ViTImageProcessor: {e}.")
    raise e

vit_transform = transforms.Compose([
    transforms.ToPILImage(), 
    transforms.Resize((processor.size["height"], processor.size["height"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])
print("Đã định nghĩa vit_transform (dựa trên ViTImageProcessor).")

# --- Đường dẫn Dữ liệu (trên Kaggle) ---
CELEBA_ROOT_DIR = "/kaggle/input/celeba-spoofing/CelebA_Spoof"
CELEBA_TEST_META = os.path.join(CELEBA_ROOT_DIR, "metas/intra_test/test_label.txt")

if not os.path.exists(CELEBA_TEST_META):
    logger.error(f"KHÔNG TÌM THẤY FILE TEST META: {CELEBA_TEST_META}")
    raise FileNotFoundError(f"Không tìm thấy file test meta: {CELEBA_TEST_META}")

# --- Tạo Test Loader ---
pad_test_dataset = CelebASpoofDataset(
    root_dir=CELEBA_ROOT_DIR, 
    meta_file=CELEBA_TEST_META, 
    transform=vit_transform # Gán transform đúng
)

pad_test_loader = DataLoader(
    pad_test_dataset, 
    batch_size=32, 
    shuffle=False, 
    num_workers=2
)

print(f"Sẵn sàng đánh giá PAD trên {len(pad_test_dataset)} mẫu test.")

Đã định nghĩa CelebASpoofDataset.


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Đã định nghĩa vit_transform (dựa trên ViTImageProcessor).
Loaded 67170 samples from /kaggle/input/celeba-spoofing/CelebA_Spoof/metas/intra_test/test_label.txt
Sẵn sàng đánh giá PAD trên 67170 mẫu test.


In [5]:
def calculate_pad_metrics(y_true, y_scores, y_preds):
    """Tính toán Acc, F1, ROC AUC, APCER, BPCER, và ACER."""
    y_true = np.array(y_true)
    y_scores = np.array(y_scores) # Xác suất spoof (lớp 1)
    y_preds = np.array(y_preds)   # Nhãn dự đoán (0 hoặc 1)
    
    # y_true: 0 = LIVE (Bona Fide), 1 = SPOOF (Attack)
    
    acc = accuracy_score(y_true, y_preds)
    f1 = f1_score(y_true, y_preds, zero_division=0)
    try:
        roc_auc = roc_auc_score(y_true, y_scores)
    except ValueError:
        roc_auc = 0.0 

    live_indices = np.where(y_true == 0)[0]
    spoof_indices = np.where(y_true == 1)[0]
    
    total_live = len(live_indices)
    total_spoof = len(spoof_indices)

    if total_live == 0 or total_spoof == 0:
        logger.warning("Tập kết quả chỉ chứa 1 lớp. Không thể tính APCER/BPCER.")
        return {"Accuracy": acc, "F1-Score": f1, "ROC AUC": roc_auc, "APCER": 0, "BPCER": 0, "ACER": 0}, None

    # BPCER: Tỷ lệ LIVE bị phân loại nhầm là SPOOF (False Positive)
    bpcer_errors = np.sum(y_preds[live_indices] == 1)
    bpcer = bpcer_errors / total_live if total_live > 0 else 0.0
    
    # APCER: Tỷ lệ SPOOF bị phân loại nhầm là LIVE (False Negative)
    apcer_errors = np.sum(y_preds[spoof_indices] == 0)
    apcer = apcer_errors / total_spoof if total_spoof > 0 else 0.0
    
    # ACER: Trung bình
    acer = (apcer + bpcer) / 2.0
    
    cm = confusion_matrix(y_true, y_preds, labels=[0, 1]) # Đảm bảo thứ tự 0, 1
    
    metrics = {
        "Accuracy": acc,
        "F1-Score": f1,
        "ROC AUC": roc_auc,
        "APCER (Tấn công bị lọt)": apcer, 
        "BPCER (Người thật bị từ chối)": bpcer, 
        "ACER (Lỗi trung bình)": acer
    }
    return metrics, cm

def plot_roc_curve_pad(y_true, y_scores, title):
    """Vẽ đường cong ROC."""
    fpr, tpr, thresholds = roc_curve(y_true, y_scores, pos_label=1)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:0.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (FPR) / BPCER')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

def plot_confusion_matrix_pad(cm, title="Confusion Matrix (PAD)"):
    """Plot the confusion matrix for Presentation Attack Detection (PAD)."""
    if cm is None or cm.shape != (2, 2):
        logger.warning(f"Invalid confusion matrix: {cm}. Skipping plot.")
        if cm is not None and cm.shape == (1, 1):
            if len(np.unique(pad_labels)) == 1 and np.unique(pad_labels)[0] == 0:
                cm = np.array([[cm[0, 0], 0], [0, 0]])
            else:
                cm = np.array([[0, 0], [0, cm[0, 0]]])
        else:
            return

    labels_axis = ['Live', 'Spoof']
    columns_axis = ['Live', 'Spoof']
    
    cm_sum = cm.sum(axis=1)[:, np.newaxis]
    cm_percent = np.nan_to_num(cm.astype('float') / cm_sum)
    hm_labels = [f"{count}\n{perc*100:0.2f}%" for count, perc in zip(cm.flatten(), cm_percent.flatten())]
    hm_labels = np.asarray(hm_labels).reshape(2, 2)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=hm_labels, fmt='', cmap='Blues',
        xticklabels=columns_axis, yticklabels=labels_axis
    )
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()


print("Đã định nghĩa các hàm tính toán và vẽ metrics PAD.")

Đã định nghĩa các hàm tính toán và vẽ metrics PAD.


In [ ]:
def run_pad_evaluation(yolo_model, vit_model, loader, transform, device):
    vit_model.eval()
    all_labels = []
    all_scores = [] # Xác suất Spoof (lớp 1)
    all_preds = []  # Nhãn dự đoán (0 hoặc 1)
    logger.info("Bắt đầu chạy đánh giá PAD trên tập test...")
    pad_threshold = 0.5 # Dùng argmax
    
    for batch_paths, batch_labels in tqdm(loader, desc="Đánh giá PAD (ViT)"):
        batch_labels = batch_labels.numpy()
        
        for idx, img_path in enumerate(batch_paths):
            true_label = batch_labels[idx]
            try:
                # 1. Đọc ảnh
                image_bgr = cv2.imread(img_path)
                if image_bgr is None: continue
                image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
                
                # 2. Chạy YOLO
                yolo_results = yolo_model(image_rgb, verbose=False, conf=0.5)
                detections = yolo_results[0].boxes
                if len(detections) == 0: continue

                # 3. Lấy box lớn nhất
                best_box = max(detections, key=lambda box: (box.xyxy[0][2]-box.xyxy[0][0])*(box.xyxy[0][3]-box.xyxy[0][1]))
                x1, y1, x2, y2 = best_box.xyxy[0].int().tolist()
                face_crop_rgb = image_rgb[y1:y2, x1:x2]
                if face_crop_rgb.shape[0] < 10 or face_crop_rgb.shape[1] < 10: continue
                
                # 4. Áp dụng transform (dùng transform của transformers)
                vit_input_tensor = transform(face_crop_rgb).unsqueeze(0).to(device)
                
                with torch.no_grad():
                    # (SỬA LỖI) Gọi model transformers
                    outputs = vit_model(vit_input_tensor) 
                    vit_logits = outputs.logits # Lấy logits từ output của transformers
                    
                    vit_probs = F.softmax(vit_logits, dim=1)
                    spoof_prob_vit = vit_probs[0, 1].item() 
                    vit_pred = vit_logits.argmax(-1).item() # Lấy argmax
                
                all_labels.append(true_label)
                all_scores.append(spoof_prob_vit)
                all_preds.append(vit_pred) 
                
            except Exception as e_process:
                logger.error(f"Lỗi khi xử lý ảnh {img_path}: {e_process}")
                continue
                
    logger.info("Hoàn tất đánh giá PAD.")
    return all_labels, all_scores, all_preds

# --- Chạy ---
pad_labels, pad_scores, pad_preds = run_pad_evaluation(
    yolo_model, 
    vit_model, 
    pad_test_loader, 
    vit_transform, # Truyền transform đúng (từ processor)
    device
)

Đánh giá PAD (ViT):   0%|          | 0/2100 [00:00<?, ?it/s]

In [ ]:
# --- Tính toán và Hiển thị Kết quả ---
if pad_labels:
    logger.info("Đang tính toán các chỉ số PAD...")
    # Tính metrics dựa trên dự đoán (pad_preds)
    # y_scores (xác suất) được dùng cho ROC AUC
    pad_metrics, pad_cm = calculate_pad_metrics(pad_labels, pad_scores, pad_preds)
    
    print("\n" + "="*60)
    print("--- 📊 KẾT QUẢ ĐÁNH GIÁ PAD (ViT TRÊN CELEBA-SPOOF TEST) ---")
    print("="*60)
    
    for metric, value in pad_metrics.items():
        print(f"{metric:<30}: {value:.4%}") # Định dạng %
    print("-"*(60))
    
    # Vẽ biểu đồ
    plot_roc_curve_pad(pad_labels, pad_scores, title="ROC Curve (ViT PAD)")
    plot_confusion_matrix_pad(pad_cm, title="Confusion Matrix (ViT PAD)")

else:
    logger.warning("Không có kết quả nào để tính toán (có thể do lỗi đọc dữ liệu hoặc YOLO không phát hiện được mặt).")